In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoporosis"
cohort = "GSE62589"

# Input paths
in_trait_dir = "../../input/GEO/Osteoporosis"
in_cohort_dir = "../../input/GEO/Osteoporosis/GSE62589"

# Output paths
out_data_file = "../../output/preprocess/Osteoporosis/GSE62589.csv"
out_gene_data_file = "../../output/preprocess/Osteoporosis/gene_data/GSE62589.csv"
out_clinical_data_file = "../../output/preprocess/Osteoporosis/clinical_data/GSE62589.csv"
json_path = "../../output/preprocess/Osteoporosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on available information, this is a SuperSeries containing gene expression data from blood samples
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait row is None since there's no explicit osteoporosis status in the sample characteristics
trait_row = None

# Age row is None since there's no age information in the sample characteristics
age_row = None

# Gender row is found at index 2, with 'Sex: female'
gender_row = 2

# 2.2 Data Type Conversion
# Since trait_row is None, no need to define convert_trait but we'll create a placeholder
def convert_trait(value):
    return None

# No age data available, but create placeholder function
def convert_age(value):
    return None

# Gender conversion - Convert to binary (0 for female, 1 for male)
def convert_gender(value):
    if value is None:
        return None
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    else:
        value = value.lower().strip()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (no clinical data available for the trait)


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Looking at the gene identifiers provided
# These are numerical identifiers (like '2315554', '2315633', etc.)
# These are not standard human gene symbols (which are typically alphabetic like 'BRCA1', 'TP53')
# These appear to be probe IDs from a microarray platform that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in gene annotation data correspond to gene IDs and gene symbols
# From the preview, we can see:
# - 'ID' column contains probe identifiers matching the gene expression data
# - 'gene_assignment' column contains gene symbol information

# 2. Get a gene mapping dataframe by extracting relevant columns
# First, check for empty gene assignments and print some examples to verify the mapping
print("Example gene assignments:")
print(gene_annotation['gene_assignment'].head(10))

# Create the mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# This handles many-to-many relationships between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Preview the resulting gene expression data
print("\nGene expression data after mapping:")
print(gene_data.head())
print(f"Shape of gene expression data: {gene_data.shape}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since trait_row is None, we can't extract proper clinical features
# Create a minimal clinical dataframe with just Gender since that's the only available feature
if gender_row is not None:
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    clinical_features_df = gender_data.T  # Transpose to get samples as rows
else:
    # If no clinical features at all, create an empty DataFrame with the same sample IDs
    clinical_features_df = pd.DataFrame(index=normalized_gene_data.columns)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# We can't handle missing values for the trait since there's no trait data
# We can only handle missing values for gender
if 'Gender' in linked_data.columns:
    # Fill missing gender values with the mode
    mode_gender = linked_data['Gender'].mode()[0] if not linked_data['Gender'].isna().all() else None
    linked_data['Gender'] = linked_data['Gender'].fillna(mode_gender)

# Since trait is not available, we can't evaluate if it's biased
# We only need to evaluate the bias in Gender
if 'Gender' in linked_data.columns:
    gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
    if gender_biased:
        print("The distribution of the feature 'Gender' in this dataset is severely biased.")
        linked_data = linked_data.drop(columns='Gender')
    else:
        print("The distribution of the feature 'Gender' in this dataset is fine.")

# 5. Conduct quality check and save the cohort information.
# Since trait_row is None, is_trait_available should be False
is_trait_available = trait_row is not None
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=is_trait_available, 
    is_biased=False,  # Set to False instead of None when trait is not available
    df=linked_data,
    note="This is a blood monocyte study. No osteoporosis status information is available in the clinical data."
)

# Since trait data is not available, the dataset is not usable for our trait analysis
print("Dataset is not usable for trait analysis due to missing trait information.")